TEST ML_composer 


In [1]:
import os
print(os.getcwd())
os.chdir("F:/project/sugarcane_disease/")
#os.chdir("O:/project/sugarcane_disease/")
os.getcwd()
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
try:
    import keras
    from keras.models import Sequential
    from tensorflow.keras import layers
    from keras.layers import MaxPooling1D, Flatten, Dense, Conv1D,MaxPooling2D, Conv2D
    from keras.layers import Dropout
    from tensorflow.keras.utils import to_categorical
    import tensorflow as tf
    import keras.metrics
except:
    try:
        from tensorflow import keras
        from tensorflow.keras import layers
        from tensorflow.keras.models import Sequential
        from tensorflow.keras.utils import to_categorical
        from tensorflow.keras.layers import MaxPooling1D, Flatten, Dense, Conv1D,MaxPooling2D, Conv2D, Dropout
        import tensorflow as tf
        from keras_self_attention import SeqSelfAttention

        print("Use tensorflow backend keras module")
    except:
        print("This is not a GPU env.")
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
from scipy import stats
import numpy as np
import configparser
from Functions import *
from tensorflow.keras.layers import Layer
from CustomLayers import *
tf.config.experimental_run_functions_eagerly(True)
# Define the residual block as a new layer

F:\PhD_Notebook\Code\ML_composer
This is not a GPU env.
Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [2]:

from GS_composer import *

This is not a GPU env.


In [25]:
from ClassModel import *
from CustomLayers import *

In [4]:
from Functions import *

In [56]:
class p_args:
    
    def __init__(self):
        
        self.ped = "sugarcane_disease"
        self.pheno = "sugarcane_disease.pheno"
        self.mpheno = 1
        self.index = "sugarcane_disease.index"
        self.model = "Attention CNN"
        self.load = None
        self.trait = "trait"
        self.output = "./results"
        self.round = 10
        self.plot = True
        self.epoch = 50
        self.rank =True
        self.lr = 0.0001
        self.batch = 2
        self.mean = False
        self.depth = 0
        self.quiet = 1
        
args = p_args()

In [20]:
config = None


In [21]:
class AttentionCNN(NN):

    def __init__(self,args):
        super(AttentionCNN,self).__init__(args)
        self.name = "Attention CNN"
        self.rank = True  ##rank block value to 0 (zero),1 (low),2 (high).
        self.args = args

    def model_name(self):
        #get class name
        return self.__class__.__name__

    def data_transform(self,geno,pheno,anno=None,pheno_standard = False):

        print("USE Attention CNN MODEL as training method")
        geno = decoding(geno)
        geno = np.expand_dims(geno, axis=2)
        #pos = np.arrays(range(geno.shape[1]))
        #pos = np.expand_dims(pos, axis=0)
        print("The transformed SNP shape:", geno.shape)

        if pheno_standard is True:
            pheno = stats.zscore(pheno)
        return geno,pheno

    def model(self, input_shape,args, optimizer="rmsprop", lr=0.00001):
        # init Q,K,V
        depth = args.depth
        input1 = layers.Input(shape=input_shape,name="input_layer_1")

        X = layers.ZeroPadding1D(padding=(0, input_shape[1]//10))(input1)
        

        V = layers.LocallyConnected1D(1,10,strides=10, activation="relu",padding="valid",use_bias=False)(X)
        V = layers.Conv1D(8,kernel_size=1,strides=1,activation='relu')(V)
        V = layers.LayerNormalization()(V)

        M = MultiHead_QK_BlockAttention()(V)

        while depth > 0:
            M = residual_fl_block(input=M, width=self.args.width, downsample=(depth % 2 == 0 & self.args.residual))
            depth -= 1
        QV_output = layers.Dense(1, activation="linear")(M)

        try:
            adm = keras.optimizers.Adam(learning_rate=lr)
            rms = keras.optimizers.RMSprop(learning_rate=lr)
            sgd = keras.optimizers.SGD(learning_rate=lr)
        except:
            adm = keras.optimizers.Adam(lr=lr)
            rms = keras.optimizers.RMSprop(lr=lr)
            sgd = keras.optimizers.SGD(lr=lr)

        optimizers = {"rmsprop": rms,
                      "Adam": adm,
                      "SGD": sgd}

        model = keras.Model(inputs=input1, outputs=QV_output)
        model.compile(optimizer=optimizers[optimizer], loss="mean_squared_error")

        #QK = layers.Dot(axes=[2, 2])([Q_encoding, K_encoding])
        #QK = layers.Softmax(axis=-1)(QK)
        #QKV = layers.Dot(axes=[2, 1])([QK, V_encoding])
        #QKV = layers.Flatten()(QKV)
        #QKV = layers.Dense(1, activation="linear")(QKV)

        return model

In [22]:
if args.output[0] == "/":
    locat = '/' + args.output.strip('/') + '/'
else:
    locat = args.output.strip('/') + '/'
if not os.path.exists(locat):
    os.mkdir(locat)

In [29]:
composer = ML_composer()
composer._model
composer.silence_mode = 2

In [57]:
#composer.get_data(config,args)
composer.args = args

In [58]:
composer.prepare_model()

In [59]:
index_ref = composer.prepare_cross_validate()

In [60]:
print("Cross-validate: {}".format(index_ref[0]))

Cross-validate: ([2, 3, 4, 5], [1])


In [61]:
train_idx,valid_idx = index_ref[0]

In [62]:
composer.prepare_training(train_idx,valid_idx)
#composer.batchSize = 64

Overall population: 4702
759 individuals need to be removed due to the miss phenotype
Filtered population: 3943
Mean of train phenotype: 4.715129635707253
Use raw phenotype as the target
16    1
39    3
44    3
47    2
92    3
Name: 2, dtype: int64


In [63]:
#composer._raw_data["GENO"].iloc[:,6:].describe()

In [64]:
composer.compose(train_idx,valid_idx)

USE Attention CNN MODEL as training method
Convert data to np.array float32
The transformed SNP shape: (3047, 26086, 1)
USE Attention CNN MODEL as training method
Convert data to np.array float32
The transformed SNP shape: (896, 26086, 1)
Train status:
Epochs:  50
Repeat(Round):  10
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer_1 (InputLayer)   [(None, 26086, 1)]        0         
_________________________________________________________________
zero_padding1d_1 (ZeroPaddin (None, 26086, 1)          0         
_________________________________________________________________
locally_connected1d_1 (Local (None, 2608, 1)           26080     
_________________________________________________________________
block_attention_1 (BlockAtte (None, 1, 2608)           13605936  
_________________________________________________________________
dense_1 (Dense)              (None, 1, 

F:\miniconda\envs\AIGS_py39\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


Epoch 1/50
 387/1524 [======>.......................] - ETA: 8:48 - loss: 1.0406

KeyboardInterrupt: 

In [65]:
#import tensorflow as tf 

if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [ ]:
1.5//1

In [ ]:
index_ref = composer.prepare_cross_validate()
i = 1
for train_idx,valid_idx in index_ref:
    print("Cross-validate: {}".format(i))
    composer.prepare_training(train_idx,valid_idx)
    composer.compose(train_idx,valid_idx)
    i+=1